In [ ]:
%load_ext autoreload
%autoreload 2

# Conditional VAE with new data from synbio_morpher

# Imports

In [ ]:
import os
import sys
from functools import partial
from datetime import datetime
from sklearn.metrics import r2_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import shuffle
import wandb
import numpy as np
import haiku as hk
import jax
import optax
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

jax.config.update('jax_platform_name', 'gpu')


# if __package__ is None:

module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)

__package__ = os.path.basename(module_path)

rng = jax.random.PRNGKey(0)

jax.devices()

In [ ]:
from src.models.vae import CVAE, sample_z
from src.models.mlp import MLP
from src.models.shared import arrayise
from src.losses.losses import loss_wrapper, compute_accuracy_regression, mse_loss, accuracy_regression
from src.utils.data_preprocessing import drop_duplicates_keep_first_n
from src.utils.math import convert_to_scientific_exponent
from src.utils.optimiser import make_optimiser
from src.utils.train import make_saves